In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_colwidth = 3000
pd.options.display.max_rows = 3000

In [19]:
df = pd.read_excel(r"/data/mmortgage/MultiPage_Classification/Bert Testing Report_12th Aug'22_Hazard Insurance & Mortgage Statement.ods")
df.head(2)

,Ref. No.,Page No.,System Classification (Python Decision),Actual Classification,Status P/F,Comments
0,ADR-2022-200000370552,417,Mortgage Statement,Mortgage Statement,Pass,NaN
1,ADR-2022-200000370552,418,Mortgage Statement,Mortgage Statement,Pass,NaN


In [20]:
df['Actual Classification'].value_counts()

Hazard Insurance                  252
Mortgage Statement                233
Credit Report                      27
Pay Off Statements                  6
Disclosures                         5
Appraisal - Appraisal Report        5
Non-Mortgage Account Statement      3
Name: Actual Classification, dtype: int64

In [21]:
df['System Classification (Python Decision)'].value_counts()

Mortgage Statement    249
Hazard Insurance      149
others                114
Others                 19
Name: System Classification (Python Decision), dtype: int64

In [22]:
df['System Classification (Python Decision)'] = df['System Classification (Python Decision)'].replace({'Others':'others'})

In [23]:
df['Status P/F'].value_counts()

Pass    359
Fail    172
Name: Status P/F, dtype: int64

In [24]:
temp = []
cats = ['Retirement Account Statement(s)',
        'Mortgage Statement',
        'Bank Statement',
        'Purchase Agreement',
        'Rental Agreements(s)',
        'Hazard Insurance',
        'Divorce Decree / Child Support',
        'Bankruptcy Papers']

for category in df['Actual Classification']:
    if category not in cats:
        temp.append('others')
    else:
        temp.append(category)
df['category'] = temp

In [25]:
df.head()

,Ref. No.,Page No.,System Classification (Python Decision),Actual Classification,Status P/F,Comments,category
0,ADR-2022-200000370552,417,Mortgage Statement,Mortgage Statement,Pass,NaN,Mortgage Statement
1,ADR-2022-200000370552,418,Mortgage Statement,Mortgage Statement,Pass,NaN,Mortgage Statement
2,ADR-2022-200000370552,419,Mortgage Statement,Mortgage Statement,Pass,NaN,Mortgage Statement
3,ADR-2022-200000370552,420,Mortgage Statement,Mortgage Statement,Pass,NaN,Mortgage Statement
4,ADR-2022-200000370552,421,Mortgage Statement,Mortgage Statement,Pass,NaN,Mortgage Statement


In [26]:
temp = []
for i,j in zip(df['category'].values, df['System Classification (Python Decision)'].values):
    if i==j:
        temp.append('PASS')
    else:
        temp.append('FAIL')
df['PF'] = temp

In [27]:
df.head()

,Ref. No.,Page No.,System Classification (Python Decision),Actual Classification,Status P/F,Comments,category,PF
0,ADR-2022-200000370552,417,Mortgage Statement,Mortgage Statement,Pass,NaN,Mortgage Statement,PASS
1,ADR-2022-200000370552,418,Mortgage Statement,Mortgage Statement,Pass,NaN,Mortgage Statement,PASS
2,ADR-2022-200000370552,419,Mortgage Statement,Mortgage Statement,Pass,NaN,Mortgage Statement,PASS
3,ADR-2022-200000370552,420,Mortgage Statement,Mortgage Statement,Pass,NaN,Mortgage Statement,PASS
4,ADR-2022-200000370552,421,Mortgage Statement,Mortgage Statement,Pass,NaN,Mortgage Statement,PASS


In [28]:
df['category'].value_counts()

Hazard Insurance      252
Mortgage Statement    233
others                 46
Name: category, dtype: int64

In [29]:
cats = ['Retirement Account Statement(s)',
        'Mortgage Statement',
        'Bank Statement',
        'Purchase Agreement',
        'Rental Agreements(s)',
        'Hazard Insurance',
        'Divorce Decree / Child Support',
        'Bankruptcy Papers',
        'others']

PASS = []
FAIL = []
cat = []
for category in cats:
    cat.append(category)
    try:
        PASS.append(df['PF'][df['category']==category].value_counts()['PASS'])
    except:
        PASS.append(None)
    
    try:
        FAIL.append(df['PF'][df['category']==category].value_counts()['FAIL'])
    except:
        FAIL.append(None)
pivot = pd.DataFrame({'category':cat,
                      'PASS': PASS,
                      'FAIL': FAIL}
                    )
pivot.dropna(thresh=2, inplace=True, axis=0)
pivot.fillna(0, inplace=True)
pivot

,category,PASS,FAIL
1,Mortgage Statement,215.0,18.0
5,Hazard Insurance,144.0,108.0
8,others,7.0,39.0


In [30]:
pivot['Accuracy'] = np.round(pivot.PASS/(pivot.PASS+pivot.FAIL),2)
pivot

,category,PASS,FAIL,Accuracy
1,Mortgage Statement,215.0,18.0,0.92
5,Hazard Insurance,144.0,108.0,0.57
8,others,7.0,39.0,0.15


In [17]:
df['Actual Classification'][(df['category']=='others')&(df['PF']=='FAIL')].value_counts()

Appraisal - Appraisal Report    17
Disclosures                     16
1003                            14
Credit Report                   12
Due Diligence Report            10
Check(s)                         6
DU Findings                      5
Power of Attorney                3
Property - Other                 2
Condo/Coop/HOA                   2
Property – Other                 2
Title - Other                    2
Credit - Other                   2
Income - Other                   1
Name: Actual Classification, dtype: int64

In [18]:
import multiprocessing
cores = multiprocessing.cpu_count()
cores

6